## Model Compilation Jupyter Notebook Example

This notebook shows the example of model compilation using edgeai-benchmark.

This script uses TIDL to compile a model and output in a format that edgeai-sdk can understand.

edgeai_benchmark is a python package provided in edgeai-benchmark that provides several functions to assist model compilation.

In [ ]:
import os
import tempfile
import argparse
import cv2
import yaml
import shutil
from edgeai_benchmark import *

In [ ]:
# the cwd must be the root of the respository
if os.path.split(os.getcwd())[-1] in ('scripts', 'tutorials'):
    os.chdir('../')
#

print(f"INFO: current dir is: {os.getcwd()}")

In [ ]:
assert ('TIDL_TOOLS_PATH' in os.environ and 'LD_LIBRARY_PATH' in os.environ and
        'TARGET_SOC' in os.environ), "Check the environment variables"
print("TIDL_TOOLS_PATH=", os.environ['TIDL_TOOLS_PATH'])
print("LD_LIBRARY_PATH=", os.environ['LD_LIBRARY_PATH'])
print("TARGET_SOC=", os.environ['TARGET_SOC'])

#### Create a temporary directory. 

This is were the compiled artifacts will be stored.

In [ ]:
modelartifacts_tempdir_name = os.path.abspath('./work_dirs_custom')
modelartifacts_custom = os.path.join(modelartifacts_tempdir_name, 'modelartifacts')
print(f'INFO: clearing modelartifacts: {modelartifacts_custom}')
if os.path.exists(modelartifacts_custom):
    shutil.rmtree(modelartifacts_custom, ignore_errors=True)
#

#### Read settings from settings_import_on_pc.yaml

Modify the settings as necessary in the constructor of settings.ConfigSettings()

In [ ]:
settings = config_settings.CustomConfigSettings('./settings_import_on_pc.yaml',
                target_device = os.environ['TARGET_SOC'], target_machine = constants.TARGET_MACHINE_PC_EMULATION,
                modelartifacts_path=modelartifacts_custom,
                model_selection=None, model_shortlist=None,
                num_frames=100)

work_dir = os.path.join(settings.modelartifacts_path, f'{settings.tensor_bits}bits')

#### Download dataset if it doesn't exist

In [ ]:
# download dataset if it doesn't exist
dataset_name='imagenetv2c'
if not os.path.exists(f'{settings.datasets_path}/{dataset_name}'):
    print(f'INFO: downloading the dataset - {dataset_name}')
    interfaces.run_download_dataset(settings, dataset_name=dataset_name)
#

#### Create Dataset Reader classes

Change the dataset paths according to your dataset location

In [ ]:
dataset_calib_cfg = dict(
    path=f'{settings.datasets_path}/imagenetv2c/val',
    split=f'{settings.datasets_path}/imagenetv2c/val.txt',
    num_classes=1000,
    shuffle=True,
    num_frames=min(settings.calibration_frames,50000),
    name='imagenet'
)

# dataset parameters for actual inference
dataset_val_cfg = dict(
    path=f'{settings.datasets_path}/imagenetv2c/val',
    split=f'{settings.datasets_path}/imagenetv2c/val.txt',
    num_classes=1000,
    shuffle=True,
    num_frames=min(settings.num_frames,50000),
    name='imagenet'
)

calib_dataset = datasets.ImageClassification(**dataset_calib_cfg)
val_dataset = datasets.ImageClassification(**dataset_val_cfg)

#### Session runtime_options

The default runtime_options can be overriden by passing a runtime_options dict to this function

In [ ]:
# choose one session_name depending on the model type
# tflitert for tflite models, onnxrt for onnx model
#session_name = constants.SESSION_NAME_TFLITERT
session_name = constants.SESSION_NAME_ONNXRT

session_type = settings.get_session_type(session_name)
runtime_options = settings.get_runtime_options(session_name, is_qat=False)

print(session_type)
print(runtime_options)

In [ ]:
preproc_transforms = preprocess.PreProcessTransforms(settings)
postproc_transforms = postprocess.PostProcessTransforms(settings)

# these session cfgs also has some default input mean and scale. 
# if your model needs a difference mean and scale, update the session cfg dict being used with those values
onnx_session_cfg = sessions.get_onnx_session_cfg(settings, work_dir=work_dir)
tflite_session_cfg = sessions.get_tflite_session_cfg(settings, work_dir=work_dir)

#### Create pipeline_configs

pipeline_configs is nothing but a dict with the various model configs that we want to compile

In [ ]:
pipeline_configs = {
    'cl-mnv2': dict(
        task_type='classification',
        calibration_dataset=calib_dataset,
        input_dataset=val_dataset,
        preprocess=preproc_transforms.get_transform_onnx(),
        session=session_type(**onnx_session_cfg,
            runtime_options=runtime_options,
            model_path=f'{settings.models_path}/vision/classification/imagenet1k/torchvision/mobilenet_v2_tv.onnx'),
        postprocess=postproc_transforms.get_transform_classification(),
        model_info=dict(metric_reference={'accuracy_top1%':71.88})
    ),
}
print(pipeline_configs)

#### Model Compilation

This will take a few minutes. Please be patient...

In [ ]:
# run the model compliation/import and inference
interfaces.run_benchmark_config(settings, work_dir, pipeline_configs)
run_dir = list(pipeline_configs.values())[0]['session'].get_run_dir()
print(f"\nINFO: compiled artifacts is in: {run_dir}")

# print the result:
with open(os.path.join(run_dir, "result.yaml")) as fp:
    result_yaml = yaml.safe_load(fp)
    result_dict = result_yaml['result']
    print(f"INFO: result - {result_dict}")
#

#### Package artifacts

Package the artifacts into a .tar.gz file, keeping only the necessary files for inference.

In [ ]:
print()
out_dir = f'{work_dir}_package'
interfaces.package_artifacts(settings, work_dir, out_dir)

#### Download

Download the packaged .tar.gz artifact

TODO: add a download link here, that the user can click to download the packaged artifact .tar.gz file.

In [ ]:
print(f'\nINFO: download the atricats files from the folder: {out_dir}')
print(os.listdir(out_dir))